# Tables OmniParser
This notebook is meant for testing and development of an "OmniParser" that can parse all components of an HTML document, including the title, captions, sentences, tables, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser
doc_parser = HTMLParser(path='data/diseases/diseases.xhtml')
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser)

In [5]:
# def parse_tag(tag):
#     if tag.name is not None:
#         print tag.name
#         print "---"
#         for child in tag.children:
#             parse_tag(child)

# from bs4 import BeautifulSoup
# with open('data/diseases/diseases.xhtml','r') as f:
#     soup = BeautifulSoup(f, 'lxml')
# for tag in soup.children:
#     parse_tag(tag)


In [ ]:
%time corpus = cp.parse_corpus(name='Diseases Corpus')

[document]
html
head
body
h1
p
br
p
span
h1
h2
TABLE!
> /Users/bradenhancock/snorkel/snorkel/parser.py(274)parse_tag()
-> for phrase in self.table_parser.parse(document, unicode(child)):


In [4]:
print len(corpus.documents)
print len(corpus.documents[0].tables)
print len(corpus.documents[0].tables[0].phrases) + len(corpus.documents[0].tables[1].phrases)
print len(corpus.documents[0].phrases)

1
0


IndexError: list index out of range

In [ ]:
from load_dictionaries import load_disease_dictionary

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

In [ ]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import DictionaryMatch

# Define a candidate space
table_ngrams = TableNgrams(n_max=3)

# Define a matcher
disease_matcher = DictionaryMatch(d=diseases, longest_match_only=False)

In [ ]:
# With old Candidates object:
from snorkel.candidates import EntityExtractor
ce = EntityExtractor(table_ngrams, disease_matcher)
%time candidates = ce.extract(corpus.get_tables(), name='all')

for cand in candidates: print cand

In [ ]:
c = candidates[0]
for ngram in c.row_ngrams('words'): print ngram

In [ ]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer()
featurizer.fit_transform(candidates)

In [ ]:
featurizer.get_features_by_candidate(candidates[1])[:]

The end.